In [2]:
import random
import sys
from itertools import chain

import numpy as np

from bounding_box_classifiers.obama_detector import main

In [3]:
params = {
    "--lr": [0.1, 0.01, 0.001, 0.001],
    "--momentum": [0.5, 0.6, 0.7, 0.8, 0.9],
    "--batch-size": [32, 64, 128]
}
used_params = set()

param_space_size = np.prod([len(p) for p in params])
while len(used_params) < param_space_size:
    curr_params = {k: str(random.choice(v)) for k, v in params.items()}
    curr_params = tuple(chain.from_iterable(curr_params.items()))
    if curr_params not in used_params:
        print(curr_params)
        used_params.add(curr_params)
        sys.argv = [
            'obama_detector',  # not used by script
            'data/bounding-box-obama-detector',
            *curr_params
        ]
        main()

('--lr', '0.01', '--momentum', '0.9', '--batch-size', '64')
Train Epoch: 1 [0/248 (0%)]	Loss: 0.717204

Test set: Average loss: 3.0733, Accuracy: 39/64 (61%)

Train Epoch: 2 [0/248 (0%)]	Loss: 5.792525

Test set: Average loss: 23.3720, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 30.825083

Test set: Average loss: 8.0457, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 2.601787

Test set: Average loss: 0.8114, Accuracy: 31/64 (48%)

Train Epoch: 5 [0/248 (0%)]	Loss: 0.698038

Test set: Average loss: 0.7108, Accuracy: 31/64 (48%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.678740

Test set: Average loss: 1.7493, Accuracy: 32/64 (50%)

Train Epoch: 7 [0/248 (0%)]	Loss: 0.691721

Test set: Average loss: 3.5384, Accuracy: 32/64 (50%)

Train Epoch: 8 [0/248 (0%)]	Loss: 0.702648

Test set: Average loss: 2.7111, Accuracy: 32/64 (50%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.703626

Test set: Average loss: 1.2981, Accuracy: 29/64 (45%)

Train Epoch: 10 [0/248 (0%)]	Loss: 0.702235

Te

Train Epoch: 1 [0/248 (0%)]	Loss: 0.717204

Test set: Average loss: 23872.4297, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 11.319605

Test set: Average loss: 492.3880, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 2.328845

Test set: Average loss: 56.4814, Accuracy: 31/64 (48%)

Train Epoch: 4 [0/248 (0%)]	Loss: 0.710494

Test set: Average loss: 8.3676, Accuracy: 30/64 (47%)

Train Epoch: 5 [0/248 (0%)]	Loss: 0.692760

Test set: Average loss: 1.6627, Accuracy: 27/64 (42%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.679867

Test set: Average loss: 0.7432, Accuracy: 29/64 (45%)

Train Epoch: 7 [0/248 (0%)]	Loss: 0.691404

Test set: Average loss: 0.6951, Accuracy: 32/64 (50%)

Train Epoch: 8 [0/248 (0%)]	Loss: 0.686664

Test set: Average loss: 0.6938, Accuracy: 31/64 (48%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.690575

Test set: Average loss: 0.6936, Accuracy: 30/64 (47%)

Train Epoch: 10 [0/248 (0%)]	Loss: 0.692057

Test set: Average loss: 0.6934, Accuracy: 29/64 (45%)

T

Train Epoch: 1 [0/248 (0%)]	Loss: 0.703073

Test set: Average loss: 0.7213, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 3.946356

Test set: Average loss: 0.6969, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 1.098757

Test set: Average loss: 0.7971, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 4.670131

Test set: Average loss: 0.7212, Accuracy: 32/64 (50%)

Train Epoch: 5 [0/248 (0%)]	Loss: 2.570156

Test set: Average loss: 0.6952, Accuracy: 32/64 (50%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.788294

Test set: Average loss: 0.6876, Accuracy: 32/64 (50%)

Train Epoch: 7 [0/248 (0%)]	Loss: 1.097162

Test set: Average loss: 1.0624, Accuracy: 32/64 (50%)

Train Epoch: 8 [0/248 (0%)]	Loss: 4.185573

Test set: Average loss: 1.2655, Accuracy: 32/64 (50%)

Train Epoch: 9 [0/248 (0%)]	Loss: 4.800737

Test set: Average loss: 0.9416, Accuracy: 32/64 (50%)

Train Epoch: 10 [0/248 (0%)]	Loss: 2.213675

Test set: Average loss: 1.5737, Accuracy: 32/64 (50%)

Train Epo

Train Epoch: 1 [0/248 (0%)]	Loss: 0.717204

Test set: Average loss: 0.9966, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 11.182215

Test set: Average loss: 1.0142, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 8.700953

Test set: Average loss: 0.8665, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 2.540831

Test set: Average loss: 1.3670, Accuracy: 30/64 (47%)

Train Epoch: 5 [0/248 (0%)]	Loss: 3.249235

Test set: Average loss: 2.7838, Accuracy: 32/64 (50%)

Train Epoch: 6 [0/248 (0%)]	Loss: 7.249365

Test set: Average loss: 1.5913, Accuracy: 34/64 (53%)

Train Epoch: 7 [0/248 (0%)]	Loss: 2.481964

Test set: Average loss: 2.0025, Accuracy: 31/64 (48%)

Train Epoch: 8 [0/248 (0%)]	Loss: 1.389712

Test set: Average loss: 1.3481, Accuracy: 31/64 (48%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.896176

Test set: Average loss: 1.1420, Accuracy: 32/64 (50%)

Train Epoch: 10 [0/248 (0%)]	Loss: 1.079113

Test set: Average loss: 1.1061, Accuracy: 32/64 (50%)

Train Ep

Train Epoch: 1 [0/248 (0%)]	Loss: 0.717204

Test set: Average loss: 1.0137, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 11.545488

Test set: Average loss: 1.2566, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 11.460459

Test set: Average loss: 0.7348, Accuracy: 30/64 (47%)

Train Epoch: 4 [0/248 (0%)]	Loss: 1.531381

Test set: Average loss: 2.3858, Accuracy: 32/64 (50%)

Train Epoch: 5 [0/248 (0%)]	Loss: 7.172695

Test set: Average loss: 3.0410, Accuracy: 32/64 (50%)

Train Epoch: 6 [0/248 (0%)]	Loss: 8.217239

Test set: Average loss: 1.6092, Accuracy: 30/64 (47%)

Train Epoch: 7 [0/248 (0%)]	Loss: 3.400250

Test set: Average loss: 1.6881, Accuracy: 30/64 (47%)

Train Epoch: 8 [0/248 (0%)]	Loss: 3.100624

Test set: Average loss: 2.8043, Accuracy: 32/64 (50%)

Train Epoch: 9 [0/248 (0%)]	Loss: 4.189847

Test set: Average loss: 2.7964, Accuracy: 32/64 (50%)

Train Epoch: 10 [0/248 (0%)]	Loss: 3.924377

Test set: Average loss: 2.5227, Accuracy: 32/64 (50%)

Train E

...

Train Epoch: 1 [0/248 (0%)]	Loss: 0.703073

Test set: Average loss: 1137.2332, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 428.290833

Test set: Average loss: 100235.0938, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 97.423836

Test set: Average loss: 13263.0039, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 2.380872

Test set: Average loss: 2472.4329, Accuracy: 32/64 (50%)

Train Epoch: 5 [0/248 (0%)]	Loss: 0.718964

Test set: Average loss: 514.1741, Accuracy: 32/64 (50%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.699527

Test set: Average loss: 92.3707, Accuracy: 33/64 (52%)

Train Epoch: 7 [0/248 (0%)]	Loss: 0.699271

Test set: Average loss: 13.9068, Accuracy: 32/64 (50%)

Train Epoch: 8 [0/248 (0%)]	Loss: 0.687941

Test set: Average loss: 4.8849, Accuracy: 36/64 (56%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.691064

Test set: Average loss: 2.2535, Accuracy: 33/64 (52%)

Train Epoch: 10 [0/248 (0%)]	Loss: 0.687034

Test set: Average loss: 1.2252, Accuracy: 

Train Epoch: 1 [0/248 (0%)]	Loss: 0.767255

Test set: Average loss: 19865.0513, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 0.796897

Test set: Average loss: 2499.7865, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 8.276668

Test set: Average loss: 303.5102, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 0.684658

Test set: Average loss: 56.6435, Accuracy: 32/64 (50%)

Train Epoch: 5 [0/248 (0%)]	Loss: 0.714770

Test set: Average loss: 3.6827, Accuracy: 27/64 (42%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.683558

Test set: Average loss: 0.6931, Accuracy: 32/64 (50%)

Train Epoch: 7 [0/248 (0%)]	Loss: 0.693147

Test set: Average loss: 0.6934, Accuracy: 32/64 (50%)

Train Epoch: 8 [0/248 (0%)]	Loss: 0.699701

Test set: Average loss: 0.6932, Accuracy: 32/64 (50%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.691868

Test set: Average loss: 0.6935, Accuracy: 32/64 (50%)

Train Epoch: 10 [0/248 (0%)]	Loss: 0.698796

Test set: Average loss: 0.6933, Accuracy: 32/64 (50%)


Train Epoch: 1 [0/248 (0%)]	Loss: 0.703073

Test set: Average loss: 4.9067, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 92.781914

Test set: Average loss: 11.5957, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 31.270893

Test set: Average loss: 10.7353, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 10.413653

Test set: Average loss: 24.1038, Accuracy: 32/64 (50%)

Train Epoch: 5 [0/248 (0%)]	Loss: 20.002451

Test set: Average loss: 12.9048, Accuracy: 32/64 (50%)

Train Epoch: 6 [0/248 (0%)]	Loss: 6.238681

Test set: Average loss: 5.9849, Accuracy: 31/64 (48%)

Train Epoch: 7 [0/248 (0%)]	Loss: 6.813680

Test set: Average loss: 9.9642, Accuracy: 32/64 (50%)

Train Epoch: 8 [0/248 (0%)]	Loss: 7.263302

Test set: Average loss: 3.6289, Accuracy: 35/64 (55%)

Train Epoch: 9 [0/248 (0%)]	Loss: 4.045724

Test set: Average loss: 4.6108, Accuracy: 31/64 (48%)

Train Epoch: 10 [0/248 (0%)]	Loss: 4.274792

Test set: Average loss: 1.7464, Accuracy: 33/64 (52%)

T

Train Epoch: 1 [0/248 (0%)]	Loss: 0.717204

Test set: Average loss: 92271.3984, Accuracy: 32/64 (50%)

Train Epoch: 2 [0/248 (0%)]	Loss: 63.209484

Test set: Average loss: 9211.2988, Accuracy: 32/64 (50%)

Train Epoch: 3 [0/248 (0%)]	Loss: 6.209377

Test set: Average loss: 2834.0159, Accuracy: 32/64 (50%)

Train Epoch: 4 [0/248 (0%)]	Loss: 0.713907

Test set: Average loss: 778.7473, Accuracy: 32/64 (50%)

Train Epoch: 5 [0/248 (0%)]	Loss: 0.843131

Test set: Average loss: 129.4495, Accuracy: 32/64 (50%)

Train Epoch: 6 [0/248 (0%)]	Loss: 0.702124

Test set: Average loss: 10.8642, Accuracy: 32/64 (50%)

Train Epoch: 7 [0/248 (0%)]	Loss: 0.696297

Test set: Average loss: 0.7022, Accuracy: 35/64 (55%)

Train Epoch: 8 [0/248 (0%)]	Loss: 0.675684

Test set: Average loss: 0.6867, Accuracy: 32/64 (50%)

Train Epoch: 9 [0/248 (0%)]	Loss: 0.714679

Test set: Average loss: 0.6871, Accuracy: 32/64 (50%)

Train Epoch: 10 [0/248 (0%)]	Loss: 0.703696

Test set: Average loss: 0.6875, Accuracy: 32/64 

KeyboardInterrupt: 

# Best results
Both with 62% test accuracy:
```
('--lr', '0.001', '--momentum', '0.9', '--batch-size', '32')  # epoch 2
('--lr', '0.01', '--momentum', '0.6', '--batch-size', '32')  # epoch 5
```